In [1]:
# prompt: conectar al drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#install librerias


In [ ]:
!pip install rasterio
!python /content/drive/MyDrive/UPCProyecto/utils/divide_mask.py

#2m
#Total mask tiles saved: 22967
#Total empty tiles: 39533

In [ ]:
!python /content/drive/MyDrive/UPCProyecto/utils/divide_img.py


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm


In [3]:
import sys
import os

sys.path.append("/content/drive/MyDrive/UPCProyecto/") # Assuming 'utils' is in 'src'

from utils.dataset import SegmentationDataset, get_transforms # Now you should be able to import
from model import UNet
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
num_classes = 12
batch_size = 20
epochs = 10
lr = 0.0001

# Dataset y DataLoader
train_images_dir = "/content/drive/MyDrive/UPCProyecto/Data/img368"
train_masks_dir = "/content/drive/MyDrive/UPCProyecto/Data/mask368"
train_dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform=get_transforms())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [ ]:

# Normalización de pesos para la función de pérdida (ignorando clases 0)
class_weights = torch.ones(num_classes).to(device)
class_weights[0] = 0  # Clase "mar" ignorada

In [ ]:



criterion = nn.CrossEntropyLoss(weight=class_weights, ignore_index=0)  # Ignoramos la clase 0

# Modelo y optimizador
model = UNet(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Scheduler para reducir el learning rate si la pérdida no mejora
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# Función de entrenamiento
def train():
    best_loss = float('inf')  # Para rastrear la mejor pérdida

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        class_correct = torch.zeros(num_classes).to(device)
        class_total = torch.zeros(num_classes).to(device)

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
        for images, masks in loop:
            images, masks = images.to(device), masks.to(device)

            # Convertir la clase 41 en 0 para que sea ignorada
            #masks[masks == 41] = 0

            optimizer.zero_grad()
            outputs = model(images)

            # Calcular pérdida
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Calcular accuracy por clase (sin contar clases 0 y 41)
            _, predicted = torch.max(outputs, 1)
            for i in range(num_classes):
                if i not in [0]:  # Ignorar clases no deseadas
                    class_correct[i] += (predicted[masks == i] == i).sum().item()
                    class_total[i] += (masks == i).sum().item()

            # Actualizar barra de progreso
            loop.set_postfix(loss=loss.item())

        # Resultados de la época
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

        for i in range(num_classes):
            if i not in [0] and class_total[i] > 0:
                acc = 100 * class_correct[i] / class_total[i]
                print(f"Clase {i}: {acc:.2f}% accuracy")

        # Ajustar el learning rate si la loss no mejora
        scheduler.step(epoch_loss)

        # Guardar el mejor modelo
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            name = f"model_{epoch}_{epoch_loss:.4f}.pth"
            torch.save(model.state_dict(), name)
            print(f"Mejor modelo guardado con Loss: {epoch_loss:.4f}")

train()

Epoch 1/10: 100%|██████████| 57/57 [14:39<00:00, 15.43s/it, loss=1.34]


Epoch 1/10, Loss: 1.6436
Clase 1: 0.00% accuracy
Clase 2: 0.46% accuracy
Clase 3: 92.86% accuracy
Clase 4: 0.26% accuracy
Clase 5: 0.00% accuracy
Clase 6: 0.00% accuracy
Clase 7: 0.00% accuracy
Clase 8: 0.00% accuracy
Clase 9: 0.00% accuracy
Clase 10: 0.02% accuracy
Clase 11: 7.54% accuracy
Mejor modelo guardado con Loss: 1.6436


Epoch 2/10: 100%|██████████| 57/57 [04:59<00:00,  5.25s/it, loss=1.38]


Epoch 2/10, Loss: 1.3572
Clase 1: 57.05% accuracy
Clase 2: 0.62% accuracy
Clase 3: 91.01% accuracy
Clase 4: 0.14% accuracy
Clase 5: 0.00% accuracy
Clase 6: 0.00% accuracy
Clase 7: 0.00% accuracy
Clase 8: 0.00% accuracy
Clase 9: 0.00% accuracy
Clase 10: 0.00% accuracy
Clase 11: 0.00% accuracy
Mejor modelo guardado con Loss: 1.3572


Epoch 3/10: 100%|██████████| 57/57 [04:56<00:00,  5.20s/it, loss=1.13]


...

Epoch 4/10: 100%|██████████| 57/57 [04:57<00:00,  5.22s/it, loss=1.13]


Epoch 4/10, Loss: 1.1926
Clase 1: 82.19% accuracy
Clase 2: 0.39% accuracy
Clase 3: 90.02% accuracy
Clase 4: 0.97% accuracy
Clase 5: 0.00% accuracy
Clase 6: 0.00% accuracy
Clase 7: 0.00% accuracy
Clase 8: 0.00% accuracy
Clase 9: 0.00% accuracy
Clase 10: 0.00% accuracy
Clase 11: 0.00% accuracy
Mejor modelo guardado con Loss: 1.1926


Epoch 5/10: 100%|██████████| 57/57 [04:55<00:00,  5.18s/it, loss=1.1]


Epoch 5/10, Loss: 1.1457
Clase 1: 81.86% accuracy
Clase 2: 0.36% accuracy
Clase 3: 81.32% accuracy
Clase 4: 29.84% accuracy
Clase 5: 0.00% accuracy
Clase 6: 49.31% accuracy
Clase 7: 0.00% accuracy
Clase 8: 0.00% accuracy
Clase 9: 2.08% accuracy
Clase 10: 0.00% accuracy
Clase 11: 0.00% accuracy
Mejor modelo guardado con Loss: 1.1457


Epoch 6/10:  14%|█▍        | 8/57 [00:41<04:15,  5.20s/it, loss=1.24]